# MVD 6. cvičení

## 1. část - PageRank

Data k dnešnímu cvičení použijte z tohoto [Github repozitáře](https://github.com/chonyy/PageRank-HITS-SimRank/tree/master/dataset). Konkrétně nás budou zajímat soubory *graph_1.txt* až *graph_6.txt*. K daným datasetům je v repozitáři implementace PageRank algoritmu, každopádně se touto implementací nijak neinspirujte. 

Cílem je naprogramovat PageRank vektorizovaně podle přednášky, povoleno je pouze použití knihovny numpy. Parametr $\alpha$ nastavte na hodnotu 0.2 a počet iterací bude 100. U prvních grafů uvidíte, že PageRank konverguje mnohem dříve a u těch složitějších nemusí stačit ani 100 iterací.

<br>
$
p^{(0)} = (\frac{1}{N}, ..., \frac{1}{N})^T \\
A = ((1-\alpha)M + \frac{\alpha}{N}E) \\
Opakujte: \\
\hspace{1cm}p^{(i+1)} = A^Tp^{(i)}
$

Pozor: Stránka, která nemá výstupní linky, musí mít nastavený parametr $\alpha$ na 1.

In [1]:
import numpy as np

In [2]:
def read_file(path:str) -> tuple[list,int]:
    size = 0
    out = {}
    with open(path) as f:
        for line in f:
            verticles = line.replace("\n","").split(",")
            if verticles[0] in out:
                out[verticles[0]].append(int(verticles[1]))
            else:
                out[verticles[0]] = [int(verticles[1])]

            if max(int(verticles[0]),int(verticles[1])) > size: #size of graph
                size = max(int(verticles[0]),int(verticles[1]))
    return out, size

In [3]:
def prob_matrix(out:list, size:int) -> np.array:
    M = np.zeros((size,size))
    for verticle, edge in zip(out.keys(),out.values()): # probability matrix
        M[np.array(int(verticle)) -1,np.array(edge)-1] = 1/len(edge)

    return M

In [4]:
def page_rank(M:np.array,
              size:int,
              alpha:float,
              it:int
              ) -> np.array:

    pd = np.zeros((size,size))
    mask = M.any(axis=1) # nalézt prázdné řádky (tedy žádná výstupní hrana)
    alpha_vec = np.where(mask, alpha, 1) # pro tyto řádky alpha = 1

    pd[...] = 1/size
    first = ((1-alpha_vec) * M.T).T
    second = (alpha_vec * pd.T).T # transpozice kvůli sloupcovému vektoru
    A = first + second

    pn = A.T @ np.array(size * [1/size])[:,None]

    for i in range(it):
        pn = A.T @ pn

    return pn


In [5]:
alpha = 0.2
it = 100
data = []
data.append(read_file("dataset/graph_1.txt"))
data.append(read_file("dataset/graph_2.txt"))
data.append(read_file("dataset/graph_3.txt"))
data.append(read_file("dataset/graph_4.txt"))
data.append(read_file("dataset/graph_5.txt"))
data.append(read_file("dataset/graph_6.txt"))
for i,(out,size) in enumerate(data):
    print("##### graph_" + str(i+1) + " ######")
    M = prob_matrix(out,size)
    pd = page_rank(M,size,alpha, it)
    print(pd)


##### graph_1 ######
[[0.0656044 ]
 [0.11808792]
 [0.16007474]
 [0.19366419]
 [0.22053575]
 [0.242033  ]]
##### graph_2 ######
[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
##### graph_3 ######
[[0.17857143]
 [0.32142857]
 [0.32142857]
 [0.17857143]]
##### graph_4 ######
[[0.27257372]
 [0.15666713]
 [0.13837881]
 [0.10924643]
 [0.18531604]
 [0.06563464]
 [0.07218322]]
##### graph_5 ######
[[0.00163995]
 [0.00163995]
 [0.00163995]
 [0.00163995]
 [0.00163995]
 [0.00185861]
 [0.00177115]
 [0.00182737]
 [0.00196794]
 [0.00196794]
 [0.00182737]
 [0.00177115]
 [0.00196794]
 [0.00196794]
 [0.00177115]
 [0.00169711]
 [0.00170178]
 [0.0019428 ]
 [0.00198014]
 [0.00169465]
 [0.00365343]
 [0.003554  ]
 [0.00169465]
 [0.00402315]
 [0.00182136]
 [0.00185236]
 [0.00187664]
 [0.00170178]
 [0.00170178]
 [0.00240599]
 [0.00197445]
 [0.00175037]
 [0.00246671]
 [0.00225534]
 [0.00253521]
 [0.00193072]
 [0.00178986]
 [0.00168645]
 [0.00169465]
 [0.004043  ]
 [0.00170178]
 [0.00169465]
 [0.00369723]
 [0.00445807]
 [

### Předpokládaný výstup

#### graph_1.xt
```python
# array([[0.0656044 ],
#        [0.11808792],
#        [0.16007474],
#        [0.19366419],
#        [0.22053575],
#        [0.242033  ]])
```
       
#### graph_2.txt
```python
# array([[0.2],
#        [0.2],
#        [0.2],
#        [0.2],
#        [0.2]])
```


#### graph_3.txt
```python
# array([[0.17857143],
#        [0.32142857],
#        [0.32142857],
#        [0.17857143]])
```


#### graph_4.txt
```python
# array([[0.27257372],
#        [0.15666713],
#        [0.13837881],
#        [0.10924643],
#        [0.18531604],
#        [0.06563464],
#        [0.07218322]])
```


## 2. část - HITS

Použijte stejná data jako u PageRank algoritmu a počet iterací bude opět 100.

Implementujte dle následujícího algoritmu:
<br>

$
a^{(0)} = (1, ..., 1)^T, h^{(0)} = (1, ..., 1)^T
\\
Opakujte:\\
    \hspace{1cm} h^{(i+1)} = Aa^{(i)}\\
    \hspace{1cm} h^{(i+1)} = \frac{h^{(i+1)}}{||h^{(i+1)}||_1}\\
    \hspace{1cm} a^{(i+1)} = A^Th^{(i)}\\
    \hspace{1cm} a^{(i+1)} = \frac{a^{(i+1)}}{||a^{(i+1)}||_1}\\   
$

In [6]:
def adj_matrix(out:list, size:int) -> np.array:
    M = np.zeros((size,size))
    for verticle, edge in zip(out.keys(),out.values()): # probability matrix
        M[np.array(int(verticle)) -1,np.array(edge)-1] = 1
    return M

In [7]:
def hits(A:np.array,
         size:int,
         it:int
         ) -> tuple[np.array,np.array]:

    a = np.ones((size,1))
    h = np.ones((size,1))
    for i in range(it):
        h_1 = A @ a
        h_1 = h_1 / (np.linalg.norm(h_1,ord = 1) + 0.00000001)
        a_1 = A.T @ h
        a_1 = a_1 / (np.linalg.norm(a_1,ord = 1) + 0.00000001)
        a,h = a_1, h_1


    return h,a

In [8]:
for i,(out,size) in enumerate(data):
    print("##### graph_" + str(i+1) + " ######")
    A = adj_matrix(out,size)
    hub,authority = hits(A,size, it)
    print("## Authority ##")
    print(authority)

    print("## HUB ##")
    print(hub)

##### graph_1 ######
## Authority ##
[[0. ]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
## HUB ##
[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0. ]]
##### graph_2 ######
## Authority ##
[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
## HUB ##
[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
##### graph_3 ######
## Authority ##
[[0.190983  ]
 [0.30901699]
 [0.30901699]
 [0.190983  ]]
## HUB ##
[[0.190983  ]
 [0.30901699]
 [0.30901699]
 [0.190983  ]]
##### graph_4 ######
## Authority ##
[[0.13948389]
 [0.17791203]
 [0.2008232 ]
 [0.14017775]
 [0.20142536]
 [0.05608926]
 [0.08408849]]
## HUB ##
[[0.27545318]
 [0.04776231]
 [0.10868324]
 [0.19865956]
 [0.1837346 ]
 [0.11673471]
 [0.06897241]]
##### graph_5 ######
## Authority ##
[[0.00000000e+000]
 [0.00000000e+000]
 [0.00000000e+000]
 [0.00000000e+000]
 [0.00000000e+000]
 [2.61604965e-057]
 [2.68924599e-027]
 [3.52327049e-047]
 [3.86356655e-086]
 [3.86356655e-086]
 [3.52327049e-047]
 [2.68924599e-027]
 [3.86356655e-086]
 [3.86356655e-086]
 [2.68924599e-027]
 [1.47000504e-003]
 

### Předpokládaný výstup

#### graph_1.xt
```python
Authority:[[0. ]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0. ]]
```
       
#### graph_2.txt
```python
Authority:[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
```


#### graph_3.txt
```python
Authority:[[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
Hub: [[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
```


#### graph_4.txt
```python
Authority:[[0.13948389]
 [0.17791203]
 [0.20082321]
 [0.14017775]
 [0.20142536]
 [0.05608926]
 [0.08408849]]
Hub: [[0.27545318]
 [0.04776231]
 [0.10868324]
 [0.19865956]
 [0.1837346 ]
 [0.11673471]
 [0.06897241]]
```


## Bonus - Invertovaný index pomocí MapReduce

Bonusovou úlohou je vytvoření invertovaného indexu stejně, jako je uvedeno na příkladu v přednášce. Implementace nebude v standardním MapReduce frameworku, ale použijete python funkce ```map()``` a ```reduce()```. Funkci map lze poté spustit paralelně s pomocí ```Pool``` objektu z knihovny ```multiprocessing```. 

Vstupními daty budou Medium články, které jsme používali v posledních pár cvičeních. Z těchto článků použijte pouze nadpisy (title). 

In [1]:
import pandas as pd
from functools import reduce
import spacy

lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner']) # NLTK


C:\Users\podav\anaconda3\envs\data_science\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [2]:

def normalize_data(db:pd.DataFrame) -> pd.DataFrame:
    # set all to lowercase
    db['title'] = db['title'].str.lower()

    # delete all special characters
    db['title'] = db['title'].str.replace(r'\W', ' ', regex=True)
    # delete multiple spaces
    db['title'] = db['title'].str.replace(r'\s+', ' ', regex=True)
    return db

def apply_lem(db:pd.DataFrame) -> pd.DataFrame:
    for i,item in db.iterrows():
        line = " ".join([token.lemma_ for token in lemmatizer(item["title"])])
        db.loc[i, 'title'] = line
    return db

#data load and normalize
df = pd.read_csv('articles.csv')
df = df[['title']]
df_norm = normalize_data(df)
df_norm = apply_lem(df_norm)
df_norm

,title
0,chatbots be the next big thing what happen the...
1,python for data science 8 concept you may have...
2,automate feature engineering in python towards...
3,machine learn how to go from zero to hero free...
4,reinforcement learning from scratch insight datum
...,...
332,you can build a neural network in javascript e...
333,artificial intelligence ai in 2018 and beyond ...
334,spike neural network the next generation of ma...
335,surprise neuron be now more complex than we think


### Předpokládaný výstup

```python
# print(inverted_index['be'])
# print(titles[12]) # zobrazení nadpisu pro kontrolu
```
```
[('D0', 1), ('D12', 2), ('D13', 1), ('D14', 1), ('D15', 1), ('D18', 1), ('D36', 1), ('D56', 1), ('D57', 1), ('D58', 1), ('D60', 1), ('D61', 1), ('D83', 1), ('D86', 1), ('D87', 1), ('D98', 1), ('D108', 1), ('D113', 1), ('D121', 1), ('D123', 2), ('D128', 1), ('D136', 1), ('D138', 1), ('D139', 1), ('D140', 1), ('D153', 1), ('D169', 1), ('D170', 1), ('D216', 1), ('D218', 1), ('D224', 1), ('D227', 1), ('D231', 1), ('D233', 1), ('D237', 1), ('D246', 1), ('D258', 1), ('D261', 1), ('D264', 1), ('D267', 1), ('D273', 1), ('D321', 1), ('D330', 1), ('D335', 1)]
```
deep learning be go to teach we all the lesson of our life job be for machine

Výstup bude identický za předpokladu použití spacy lemmatizéru. Zároveň výstup nemusí obsahovat stejný formát indexu, postačí *(index, count)*.

In [6]:
def count_map(line:list) -> dict:
    out = {}
    uq = set(line[1].split(" "))
    for word in uq:
        out[word] = [(line[0],line[1].count(word))]
    return out

def dict_reduce(x:dict,y:dict) -> dict:
    for key,value in zip(y.keys(),y.values()):
        value = value[0]
        if key in x:
            x[key].append(value)
        else:
            x[key] = [value]
    return x


def inverted_index(df:pd.DataFrame):
    data = [df.columns.tolist()][1:] + df.reset_index().values.tolist() #dataframe to list
    data_map = list(map(count_map,data)) #map
    ii = reduce(dict_reduce, data_map) #reduce
    return ii

In [7]:
ii = inverted_index(df_norm)

In [8]:
print(ii["be"])
df_norm["title"].iloc[12]

[(0, 1), (12, 2), (13, 1), (14, 1), (15, 1), (18, 1), (36, 1), (56, 1), (57, 1), (58, 1), (60, 1), (61, 1), (83, 1), (86, 1), (87, 1), (98, 1), (108, 2), (113, 2), (121, 1), (123, 2), (128, 1), (136, 1), (138, 1), (139, 1), (140, 1), (153, 1), (169, 1), (170, 1), (216, 1), (218, 1), (224, 1), (227, 1), (231, 1), (233, 2), (237, 1), (246, 1), (258, 1), (261, 1), (264, 1), (267, 1), (273, 1), (321, 1), (330, 1), (335, 1)]


'deep learning be go to teach we all the lesson of our life job be for machine'